In [ ]:
#Import libraries
import os
import sys
import gc
import arcpy
from arcpy import env
from arcpy.sa import *
import time
import traceback
import tempfile
import math
import subprocess
import shutil

In [ ]:
#Pull in data
#Set environments
tiles = "C:/Users/amaxwel6/Dropbox/Teaching_WVU/Spatial_Analystics/website_production/Exercises/slope_failure_pred/"
dem = "D:/v_r_landslide/inputs/dem_2m.tif"


env.overwriteOutput = True
arcpy.env.pyramids = "NONE"
arcpy.env.rasterStatistics = "NONE"
arcpy.env.compression = "NONE"
env.snapRaster = "D:/v_r_landslide/inputs/dem_2m.tif"

In [ ]:
#Split tile data so that each tile is written to a separate shapefile
with arcpy.da.SearchCursor(tiles,['SHAPE@','tile_num']) as rows:
    for row in rows:   
        tnum = row[1]
        arcpy.env.workspace = "D:/v_r_landslide/shp_out"
        arcpy.env.scratchWorkspace = "D:/v_r_landslide/scratch"
        where_clause = "\"tile_num\" = " + str(tnum)
        arcpy.Select_analysis(tiles, "tile" + str(row[1]), where_clause)

In [ ]:
#List all tiles
arcpy.env.workspace = "D:/v_r_landslide/shp_out"
shp_lst = arcpy.ListFeatureClasses()   
arcpy.env.workspace = "D:/v_r_landslide/temp"
print(shp_lst)
print(len(shp_lst))

In [ ]:
#Execute subprocess code for all tiles using a for loop. 
#Feed tile name as a variable to subprocess.
for shp in shp_lst:
    name = str(shp)
    tile = [shp]
    subprocess.call(["python", "predict_to_grid_subscript.py"] + tile, shell=True)
    arcpy.CopyRaster_management("D:/v_r_landslide\out_grids", "D:/v_r_landslide/pred_out" + str(os.path.splitext(shp)[0]) + ".tif"
    print("Process complete for " + name + "!")